# 此程序对应BINet文章第四个例子，求解不同波数下的Helmholtz方程外问题

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable as v
import scipy.special as scp
import time

is_gpu = torch.cuda.is_available()
if is_gpu:
    id = 2
    torch.cuda.set_device(id)
    
#gpu_nums = torch.cuda.device_count()
#gpu_index = torch.cuda.current_device()
#print(is_gpu,gpu_nums,gpu_index)
device = torch.device('cuda' if is_gpu else 'cpu')

device = torch.device('cpu')
print(device)
torch.set_default_tensor_type('torch.DoubleTensor')
print(torch.__version__)

cpu
1.6.0+cu101


In [4]:
#设计问题区域

# k                波数，取值范围为[2,3.5]∪[4.5,6]
# k_num            随机采样的k的个数
# sample_num       边界上样本点数
# theta, r, x0, x1 辅助确定边界点坐标
# sample_x         边界样本点坐标
# sample_u_r       准确解实部
# sample_u_i       准确解虚部
# rprime           边界参数化方程在样本点处切向量
# int_n            样本点处的外法向量
# sample_xk        构造网络的输入，即样本点坐标+k的值共同作为输入
k_num = 70
k = torch.cat((torch.rand(30)*1.5+2,torch.rand(40)*1.5+4.5),0)
 
sample_num = 1000 

theta = torch.linspace(2*np.pi/sample_num,2*np.pi,sample_num).reshape(-1,1)
r = 9/20-1/9*torch.cos(5*theta)
x0,x1 = torch.cos(theta),torch.sin(theta)
sample_x = torch.cat((x0,x1),1)*r
rprime = torch.cat((-x1*r+5/9*torch.sin(5*theta)*x0,x0*r+5/9*torch.sin(5*theta)*x1),1)
int_n = rprime@torch.Tensor([[0,-1],[1,0]])
int_n = int_n/((int_n*int_n).sum(axis=1).reshape(-1,1).sqrt())

sample_xk = torch.zeros(sample_num*k_num,3)


sample_u_r = torch.zeros(k_num,sample_num)
sample_u_i = torch.zeros(k_num,sample_num)
for l in range(k_num):
    sample_u_r[l,:] = scp.hankel1(0,k[l]*torch.sqrt((sample_x[:,0])**2+sample_x[:,1]**2)).real
    sample_u_i[l,:] = scp.hankel1(0,k[l]*torch.sqrt((sample_x[:,0])**2+sample_x[:,1]**2)).imag
    sample_xk[l*sample_num:(l+1)*sample_num,0:2] = sample_x
    sample_xk[l*sample_num:(l+1)*sample_num,2] = k[l]

In [6]:
# 设置积分矩阵，对于每个k要构造相应的矩阵

# G1_r    基本解的实部
# G1_i    基本解的虚部
# A1_r    积分矩阵的实部
# A1_i    积分矩阵的虚部
# KaparR2 Kapar奇异积分格式的权重

r0 = torch.zeros(sample_num,sample_num)
n1 = torch.zeros(sample_num,sample_num)
c = torch.zeros(sample_num,sample_num)
KaparR2 = torch.zeros(sample_num,1)
KaparR2[0],KaparR2[1] = 1.825748064736159,-1.325748064736159
for i in range(sample_num):
    for j in range(sample_num):
        d = sample_x[j,:]-sample_x[i,:]
        r0[i,j] = d.norm()
        n1[i,j] = (int_n[j,:]*d).sum()
        c[i,j] = (1+KaparR2[abs(((i-j)+round(sample_num/2))%sample_num-round(sample_num/2)-1)])*((rprime[j,:]).norm())/sample_num*2*np.pi

G2_r = torch.zeros(k_num,sample_num,sample_num)
G2_i = torch.zeros(k_num,sample_num,sample_num)
for l in range(k_num):
    G2_r[l,:,:] = scp.hankel1(1,k[l]*r0).imag/4*n1/r0*k[l]
    G2_i[l,:,:] = -scp.hankel1(1,k[l]*r0).real/4*n1/r0*k[l]


A2_r = torch.zeros(k_num,sample_num,sample_num)
A2_i = torch.zeros(k_num,sample_num,sample_num)
for l in range(k_num):
    for i in range(sample_num):
        A2_r[l,i,:],A2_i[l,i,:] = -c[i,:]*G2_r[l,i,:],-c[i,:]*G2_i[l,i,:]
        A2_r[l,i,i],A2_i[l,i,i] = -1/2,0


In [13]:
# 网络结构和损失函数

# ResNet结构
# m   神经元个数
# out 输出维数
class Net(nn.Module):
  def __init__(self,m,out):
    super(Net, self).__init__()
    self.input = nn.Linear(3,m)
    self.block1=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block2=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block3=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block4=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block5=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block6=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block7=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block8=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.out = nn.Linear(m,out)
  def forward(self, x):
      x = self.input(x)
      x = self.block1(x) + x
      x = self.block2(x) + x
      x = self.block3(x) + x
      x = self.block4(x) + x
      x = self.block5(x) + x
      x = self.block6(x) + x
      x = self.block7(x) + x
      x = self.block8(x) + x
      x = self.out(x)
      return x
class Green_loss2(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,u_exact_r,u_exact_i,u_Green_r,u_Green_i):
        return torch.mean(torch.pow((u_exact_r-u_Green_r),2)) +torch.mean(torch.pow((u_exact_i-u_Green_i),2))

net1 = Net(100,2) 
Green_loss_func = Green_loss2()

In [8]:
net1 = net1.to(device)
sample_x = sample_x.to(device)
G2_r = G2_r.to(device)
G2_i = G2_i.to(device)
A2_r = A2_r.to(device)
A2_i = A2_i.to(device)
sample_u_r = sample_u_r.to(device)
sample_u_i = sample_u_i.to(device)
sample_xk = sample_xk.to(device)
KaparR2 = KaparR2.to(device)
r0 = r0.to(device)
n1 = n1.to(device)
c = c.to(device)

In [15]:
#训练过程

# optimizer   优化器
# Epoch       总训练次数
# sample_h_r  样本点处密度函数值h(x)的实部
# sample_h_i  样本点处密度函数值h(x)的虚部
# u0_r        样本点处数值解实部
# u0_i        样本点处数值解虚部
optimizer = torch.optim.Adam(net1.parameters(net1),lr=0.0005)
Epoch = 100
loss_all = np.zeros(Epoch+1)
loss = torch.Tensor([0]).to(device)
time0 = time.time()
for epoch in range(Epoch+1):
    
    if (epoch+1)%500==0:
        k = (torch.cat((torch.rand(30)*1.5+2,torch.rand(40)*1.5+4.5),0)).to(device)
        k1,k2 = (k*np.cos(theta0)).to(device) , (k*np.sin(theta0)).to(device)
        for l in range(k_num):
            sample_u_r[l,:] = (scp.hankel1(0,(k[l]*torch.sqrt((sample_x[:,0])**2+sample_x[:,1]**2)).cpu()).real).to(device)
            sample_u_i[l,:] = (scp.hankel1(0,(k[l]*torch.sqrt((sample_x[:,0])**2+sample_x[:,1]**2)).cpu()).imag).to(device)

            sample_xk[l*sample_num:(l+1)*sample_num,0:2] = sample_x
            sample_xk[l*sample_num:(l+1)*sample_num,2] = k[l]

        for l in range(k_num):
            G2_r[l,:,:] = (scp.hankel1(1,(k[l]*r0).cpu()).imag).to(device)/4*n1/r0*k[l]
            G2_i[l,:,:] = -(scp.hankel1(1,(k[l]*r0).cpu()).real).to(device)/4*n1/r0*k[l]
        for l in range(k_num):
            for i in range(sample_num):
                A2_r[l,i,:],A2_i[l,i,:] = -c[i,:]*G2_r[l,i,:],-c[i,:]*G2_i[l,i,:]
                A2_r[l,i,i],A2_i[l,i,i] = -1/2,0
    

    sample_h_r,sample_h_i = net1(sample_xk)[:,0],net1(sample_xk)[:,1]#这个方法甚至不需要内部点
    loss = torch.Tensor([0]).to(device)
    for l in range(k_num):
        u0_r = (A2_r[l,:,:]@sample_h_r[l*sample_num:(l+1)*sample_num]-A2_i[l,:,:]@sample_h_i[l*sample_num:(l+1)*sample_num])
        u0_i = (A2_r[l,:,:]@sample_h_i[l*sample_num:(l+1)*sample_num]+A2_i[l,:,:]@sample_h_r[l*sample_num:(l+1)*sample_num])
        loss = loss + Green_loss_func(sample_u_r[l,:],sample_u_i[l,:],u0_r,u0_i)*k[l]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_all[epoch] = loss.cpu().detach().numpy() 
    if epoch%100==0:
        print('loss, epoch, computation time:','%.4f'%loss.detach().numpy(),epoch,'%.4f'%(time.time()-time0))
        time0 = time.time()

loss, epoch, computation time: 1.5490 0 3.1216
loss, epoch, computation time: 1.0124 100 571.1881


In [16]:
# 在问题所在区域内部测试精度

# sample 内部采样点数
# x_in   内部采样点坐标
# u_in_r 内部点准确解实部 
# u_in_i 内部点准确解虚部 
# G_in_r 内部点对应积分矩阵实部
# G_in_i 内部点对应积分矩阵虚部

sample = 1500
theta_in = torch.rand(sample,1)*2*np.pi
r_in = torch.rand(sample,1)*(2-(9/20-1/9*torch.cos(5*theta_in)))+(9/20-1/9*torch.cos(5*theta_in))*1.01
x0_in = torch.cos(theta_in)
x1_in = torch.sin(theta_in)
x_in = (torch.cat((x0_in,x1_in),1)*r_in).to(device)
rprime = rprime.to(device)
int_n = int_n.to(device)

r0_in = (torch.zeros(sample,sample_num)).to(device)
c_in = (torch.zeros(sample,sample_num)).to(device)
for i in range(sample):
    for j in range(sample_num):
        d = sample_x[j,:]-x_in[i,:]
        r0_in[i,j] = d.norm()
        c_in[i,j] = (rprime[j,:].norm())/sample_num*2*np.pi*((d*int_n[j,:]).sum())/r0_in[i,j]


In [ ]:
# 测试精度
# k_test 在[2,6]上等距取k_test个值作为k的值，分别测试其精度
# l_all  每个k得到的相对L2误差
# u_in_r 内部点准确解实部 
# u_in_i 内部点准确解虚部 
# G_in_r 内部点对应积分矩阵实部
# G_in_i 内部点对应积分矩阵虚部
k_test = 70
kx = torch.zeros(k_test+1)
l_all = torch.zeros(k_test+1)
for i in range(k_test+1):
    k = i/k_test*7+1

    u_in_r = (scp.hankel1(0,(k*torch.sqrt((x_in[:,0])**2+x_in[:,1]**2)).cpu()).real).to(device) 
    u_in_i = (scp.hankel1(0,(k*torch.sqrt((x_in[:,0])**2+x_in[:,1]**2)).cpu()).imag).to(device) 

    G2_in_r = (scp.hankel1(1,(k*r0_in).cpu()).imag).to(device)/4*c_in*k
    G2_in_i = -(scp.hankel1(1,(k*r0_in).cpu()).real).to(device)/4*c_in*k
    x_sample_in = (torch.zeros(sample_num,3)).to(device)
    x_sample_in[:,0:2] = sample_x
    x_sample_in[:,2] = k
    sample_h_r,sample_h_i = net1(x_sample_in)[:,0],net1(x_sample_in)[:,1]

    u_green_r = -(G2_in_r@sample_h_r-G2_in_i@sample_h_i)
    u_green_i = -(G2_in_r@sample_h_i+G2_in_i@sample_h_r)
    kx[i] = k
    l_all[i] = (((u_green_r-u_in_r)**2+(u_green_i-u_in_i)**2).sum()/((u_in_r)**2+(u_in_i)**2).sum()).sqrt()
